In [1]:
from torchvision.datasets import Cityscapes
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

# Define input transformations (resize + to tensor)
input_transform = transforms.Compose([
    transforms.Resize((256, 512)),  # Resize to match your training size
    transforms.ToTensor(),
])

# Only use the fine masks with 'semantic' targets
target_transform = transforms.Compose([
    transforms.Resize((256, 512), interpolation=transforms.InterpolationMode.NEAREST)
])

# Dataset paths
root_dir = "datasets/cityscapes"

# Load train and val splits
train_dataset = Cityscapes(
    root=root_dir,
    split='train',
    mode='fine',
    target_type='semantic',
    transform=input_transform,
    target_transform=target_transform,
)

val_dataset = Cityscapes(
    root=root_dir,
    split='val',
    mode='fine',
    target_type='semantic',
    transform=input_transform,
    target_transform=target_transform,
)

# Wrap in DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

# Sanity check
image, mask = train_dataset[0]
print("Image shape:", image.shape)  # [3, 256, 512]
print("Mask shape:", mask.shape)    # [256, 512]
print("Unique mask values:", torch.unique(mask))

RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory